<div align="center">
    <h2> CSCI7000-011 Homework Assignment 5</h2>
</div>

* There is 1 problem with 4 questions. Total points = 10.
* You need to install Z3 (opam install z3) to finish this assignment. 
* Before you turn it in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).
* Fill in any place that says `Fill in here`.

## Problem 1 (Dependent Types for CoreML)

Below is a simple implementation of dependent type checking for the CoreML language. The type checker works but hasn't been tested thoroughly. The aim of this exercise is to test it on various simple functions and their dependent types. The functions for which you need to write the AST and the dependent types are at the end of this notebook. 

In [2]:
(* Abstract Syntax *)

(* CoreML type *)

(* base types *)
type base_typ =
  | TyInt
  | TyBool
  
type typ = 
  | TyRef of string * base_typ * expr (* "refined" type. e.g., {v: int | v>0} *)
  | TyArr of string * typ * typ (* Dependent arrow type. e.g., x:{v0: int | ¬(v0=0)} -> {v1: int | v1 > x} *)

and expr =
  | Var of string
  | App of expr * expr
  | Lam of string * typ * typ option * expr (* \(x:T1):T2.e or \(x:T1).e *)
  | IntConst of int
  | BoolConst of bool
  | Plus of expr * expr
  | Minus of expr * expr
  | Mult of expr * expr
  | Eq of expr * expr
  | Gt of expr * expr
  | Lt of expr * expr
  | Not of expr
  | And of expr * expr
  | Or of expr * expr
  | Ite of expr * expr * expr
  | Let of string * expr * expr
  | LetRec of string * typ * expr * expr (* let rec (x:poly_typ) = e_1 in e_2 *)

type base_typ = TyInt | TyBool


type typ = TyRef of string * base_typ * expr | TyArr of string * typ * typ
and expr =
    Var of string
  | App of expr * expr
  | Lam of string * typ * typ option * expr
  | IntConst of int
  | BoolConst of bool
  | Plus of expr * expr
  | Minus of expr * expr
  | Mult of expr * expr
  | Eq of expr * expr
  | Gt of expr * expr
  | Lt of expr * expr
  | Not of expr
  | And of expr * expr
  | Or of expr * expr
  | Ite of expr * expr * expr
  | Let of string * expr * expr
  | LetRec of string * typ * expr * expr


In [3]:
exception TypeError of string
type typbind = string * typ
type tyenv = typbind list

exception TypeError of string


type typbind = string * typ


type tyenv = typbind list


In [4]:
#require "z3";;

/Users/gkaki/.opam/4.12.0/lib/ocaml/nums.cma: loaded
/Users/gkaki/.opam/4.12.0/lib/num-top: added to search path
/Users/gkaki/.opam/4.12.0/lib/num-top/num_top.cma: loaded
/Users/gkaki/.opam/4.12.0/lib/num: added to search path
/Users/gkaki/.opam/4.12.0/lib/z3: added to search path
/Users/gkaki/.opam/4.12.0/lib/z3/z3ml.cma: loaded


In [ ]:
(*
 * Substituting expression m for Var x inside the expression e.
 * We use this only for type checking, hence the function is only
 * implemented for expressions that can occur inside type refinement. 
 *)
let rec expr_subst (m,x) e = match e with 
  | Var y when y=x -> m
  | Var y -> e
  | Plus(e1,e2) -> Plus(expr_subst (m,x) e1, expr_subst (m,x) e2)
  | Minus(e1,e2) -> Minus(expr_subst (m,x) e1, expr_subst (m,x) e2)
  | Mult(e1,e2) -> Mult(expr_subst (m,x) e1, expr_subst (m,x) e2)
  | Eq(e1,e2) -> Eq(expr_subst (m,x) e1, expr_subst (m,x) e2)
  | Lt(e1,e2) -> Lt(expr_subst (m,x) e1, expr_subst (m,x) e2)
  | Gt(e1,e2) -> Gt(expr_subst (m,x) e1, expr_subst (m,x) e2)
  | Not e -> Not (expr_subst (m,x) e)
  | And(e1,e2) -> And(expr_subst (m,x) e1, expr_subst (m,x) e2)
  | Or(e1,e2) -> Or(expr_subst (m,x) e1, expr_subst (m,x) e2)
  | _ -> e (* This is incorrect if Lam, If-then-else etc can occur as type refinements *)

In [6]:
open Z3

module Int = Z3.Arithmetic.Integer
module Bool = Z3.Boolean
type z3_expr = Z3.Expr.expr

let rec keep_some l = match l with
  | [] -> []
  | (Some x)::xs -> x::(keep_some xs)
  | None::xs -> keep_some xs
  
(*
 * Below is the function that encodes the env into Z3 and checks
 * that it is consistent (i.e., satisfiable). 
 *)
let z3_check_sat env = 
  let cfg = [("model", "true"); ("proof", "false")] in
  let ctx = (mk_context cfg) in
  (* let s_true = mk_true ctx in *)
  let s_Int = Int.mk_sort ctx in
  let s_Bool = Bool.mk_sort ctx in
  let sort_of_typ = function
    | TyInt -> s_Int | TyBool -> s_Bool in
  let mk_var (v:string) (t:base_typ) : z3_expr = 
    Z3.Expr.mk_const_s ctx v (sort_of_typ t) in
  let (var_map : (string*z3_expr) list) = keep_some @@ List.map 
      (function
        | (x,TyRef (_,t,_)) when x <> "" -> Some (x,mk_var x t)
        | (_,_) -> None) env in
        
  (* Building Z3 AST of an expression *)
  let rec z3_expr_of pred = match pred with
  | Var x -> (try List.assoc x var_map
             with Not_found -> failwith @@ "Unknown variable: "^x)
  | IntConst i -> Int.mk_numeral_i ctx i
  | BoolConst true -> Bool.mk_true ctx
  | BoolConst false -> Bool.mk_false ctx
  | Plus(e1,e2) -> Z3.Arithmetic.mk_add ctx [z3_expr_of e1; z3_expr_of e2]
  | Minus(e1,e2) -> Z3.Arithmetic.mk_sub ctx [z3_expr_of e1; z3_expr_of e2]
  | Mult(e1,e2) -> Z3.Arithmetic.mk_mul ctx [z3_expr_of e1; z3_expr_of e2]
  | Eq(e1,e2) -> Bool.mk_eq ctx (z3_expr_of e1) (z3_expr_of e2)
  | Lt(e1,e2) -> Z3.Arithmetic.mk_lt ctx (z3_expr_of e1) (z3_expr_of e2)
  | Gt(e1,e2) -> Z3.Arithmetic.mk_gt ctx (z3_expr_of e1) (z3_expr_of e2)
  | Not e -> Bool.mk_not ctx @@ z3_expr_of e
  | And(e1,e2) -> Bool.mk_and ctx [z3_expr_of e1; z3_expr_of e2]
  | Or(e1,e2) -> Bool.mk_or ctx [z3_expr_of e1; z3_expr_of e2] 
  | _ -> failwith "Not all exprs can be type refinements" in
  
  (* Get Z3 ASTs of constraints *)
  let cstrs = keep_some @@ List.map
      (function
        | (x,TyRef (v,t,BoolConst true)) -> None
        | (x,TyRef (v,t,phi)) -> Some (z3_expr_of @@ expr_subst (Var x,v) phi)
        | _ -> None) env in
        
  (* Make a solver *)
  let solver = Z3.Solver.mk_solver ctx None in
  begin
    (* assert all constraints *)
    Solver.add solver cstrs;
    (* print -- uncomment below line to see the Z3 SMT2 encoding *)
    (* Printf.printf "%s\n%!" @@ Z3.Solver.to_string solver; *)
    (* check-sat *)
    Solver.check solver []
  end
  

module Int = Z3.Arithmetic.Integer


module Bool = Z3.Boolean


type z3_expr = Z3.Expr.expr


val keep_some : 'a option list -> 'a list = <fun>


val z3_check_sat : (string * typ) list -> Z3.Solver.status = <fun>


In [7]:
(*
 * Checks if t1 <: t2 under environment env.
 *)
let rec is_subtype env t1 t2 = match t1,t2 with
  | TyRef(_,t1,_), TyRef(_,t2,_) when t1<>t2 -> false
  | TyRef(_,t1,_), TyRef(_,t2,BoolConst true) when t1=t2 -> true
  | TyRef(v1,t1,phi1), TyRef(v2,t2,phi2) 
      when v1=v2 && t1=t2 -> 
    let env' = env@[("v",TyRef(v1,t1,And(phi1,Not phi2)))] in
    (match z3_check_sat env' with
      | Z3.Solver.UNSATISFIABLE -> true
      | Z3.Solver.SATISFIABLE -> false
      | Z3.Solver.UNKNOWN -> failwith "z3 unknown")
  | _ -> failwith "All cases in is_subtype not implemented"

val is_subtype : (string * typ) list -> typ -> typ -> bool = <fun>


In [8]:
(*
 * Substituting expression e for Var "x" inside a given type.
 *)
let rec subst (e,x) = function
  | TyRef(v,t,phi) -> TyRef(v,t,expr_subst (e,x) phi)
  | TyArr(y,ty1,ty2) -> TyArr(y, subst (e,x) ty1, subst (e,x) ty2)

val subst : expr * string -> typ -> typ = <fun>


In [9]:
let trivial (t:base_typ) : typ = TyRef("v",t, BoolConst true)

val trivial : base_typ -> typ = <fun>


In [10]:
(* 
 * Project the base type out of the refined type.
 *)
let base_of (t:typ): base_typ = match t with
 | TyRef (_,bt,_) -> bt
 | TyArr (_,_,_) -> failwith "An arrow is not a base type"

val base_of : typ -> base_typ = <fun>


In [11]:
(*
 * Below is the actual type checker. It computes the type ty_e of expression e
 * under the type environment env. If tyop = Some ty, it checks 
 * if the computed type ty_e is the subtype of the expected type ty.
 *)
let rec type_check (env: tyenv) (e: expr) (tyop: typ option) : typ = match e with
  | Var x -> 
    let ty_x = try List.assoc x env 
               with Not_found -> raise @@ TypeError ("Variable "^x^" undefined") in
    let ty_e = match ty_x with
      | TyRef(v,ty,phi) -> TyRef(v,ty,And(phi,Eq(Var v, Var x)))
      | _ -> ty_x in
    (match tyop with
      | Some ty when is_subtype env ty_e ty -> ty
      | Some _ -> raise @@ TypeError "Subtype check failed at Var"
      | None -> ty_e)
  | Lam(x,ty1,ty2op,e1) -> type_check ((x,ty1)::env) e1 ty2op
  | App(e1,e2) -> 
    let ty1 = type_check env e1 None in
    let ty2 = type_check env e2 None in
    begin
      match ty1 with
      | TyArr (x, t1, t2) when is_subtype env ty2 t1 -> subst (e2,x) t2
      | TyArr (x, t1, t2) -> raise @@ TypeError "Subtyping check failed at App"
      | _ -> raise @@ TypeError "Left hand side of application is not a function type" 
    end
  | IntConst i -> 
    let ty_c = TyRef("v",TyInt, Eq(Var "v",IntConst i)) in
    (match tyop with
      | Some ty when is_subtype env ty_c ty -> ty
      | Some _ -> raise @@ TypeError "Subtype check failed at IntConst"
      | None -> ty_c)
  | BoolConst b -> 
    let ty_c = TyRef("v",TyBool, Eq(Var "v",BoolConst b)) in
    (match tyop with
      | Some ty when is_subtype env ty_c ty -> ty
      | Some _ -> raise @@ TypeError "Subtype check failed at BoolConst"
      | None -> ty_c)
  | Plus(e1,e2) | Minus (e1,e2) | Mult(e1,e2) ->
    let (ty1,ty2) = (type_check env e1 None, type_check env e2 None) in
    let ty_e = match base_of ty1, base_of ty2 with
      | TyInt, TyInt -> TyRef("v", TyInt, Eq(Var "v", e))
      | _ -> raise @@ TypeError "Arithmetic expression needs to have Int type" in
    (match tyop with
      | Some ty when is_subtype env ty_e ty -> ty
      | Some _ -> raise @@ TypeError "Subtype check failed at arith expr."
      | None -> ty_e)
  | Eq (e1,e2) | Gt (e1,e2) | Lt (e1,e2) ->
    let (ty1,ty2) = (type_check env e1 None, type_check env e2 None) in
    let ty_e = match base_of ty1, base_of ty2 with
      | TyInt, TyInt -> TyRef("v", TyBool, Eq(Var "v", e))
      | _ -> raise @@ TypeError "Comparisons of only int expressions allowed" in
    (match tyop with
      | Some ty when is_subtype env ty_e ty -> ty
      | Some _ -> raise @@ TypeError "Subtype check failed at comparison expr."
      | None -> ty_e)
  | Not e1 ->
    let ty_e = match base_of @@ type_check env e1 None with
      | TyBool -> TyRef("v", TyBool, Eq(Var "v", e))
      | _ -> raise @@ TypeError "Argument of Not has to be a boolean" in
    (match tyop with
      | Some ty when is_subtype env ty_e ty -> ty
      | Some _ -> raise @@ TypeError "Subtype check failed at Not expr."
      | None -> ty_e)
  | And(e1,e2) | Or(e1,e2) ->
    let (ty1,ty2) = (type_check env e1 None, type_check env e2 None) in
    let ty_e = match base_of ty1, base_of ty2 with
      | TyBool, TyBool -> TyRef("v", TyBool, Eq(Var "v", e))
      | _ -> raise @@ TypeError "And/Or need boolean arguments" in
    (match tyop with
      | Some ty when is_subtype env ty_e ty -> ty
      | Some _ -> raise @@ TypeError "Subtype check failed at arith expr."
      | None -> ty_e)
  | Ite(e1,e2,e3) -> 
    let ty1 = type_check env e1 None in
    let _ = match base_of ty1 with
      | TyBool -> ()
      | _ -> raise @@ TypeError "If scrutinee needs to have boolean type" in
    let ty2 = type_check (("",TyRef("v",TyBool, Eq(e1,BoolConst true)))::env) e2 tyop in
    let ty3 = type_check (("",TyRef("v",TyBool, Eq(e1,BoolConst false)))::env) e3 tyop in
    (match ty2,ty3 with
      | TyRef(v1,bt1,phi1), TyRef(v2,bt2,phi2) when bt1<>bt2 ->
          raise @@ TypeError "Then and else branches have different base types"
      | TyRef(v1,bt1,phi1), TyRef(v2,bt2,phi2) when v1=v2 && bt1=bt2 ->  TyRef(v1,bt1,Or(phi1,phi2))
      | _,_ -> failwith "If-then-else case not implemented")
  | Let(x,e1,e2) -> 
    let ty1 = type_check env e1 None in
    type_check ((x,ty1)::env) e2 tyop
  | LetRec (x,ty_x,e1,e2) ->
    let env' = (x,ty_x)::env in
    let _ = type_check env' e1 None in
    type_check env' e2 None

val type_check : tyenv -> expr -> typ option -> typ = <fun>


## Example ##

Consider the max function:
```ocaml
let max x y = if x > y then x else y
```
Let's say we want to verify it against the type `x:int -> y:int -> {v:int | v≥x ∧ v≥y}`. Here is how you do it:

In [ ]:
let max_ret_ty = TyRef("v",TyInt,And(Or(Gt(Var "v", Var "x"), Eq(Var "v", Var "x")), 
                                     Or(Gt(Var "v", Var "y"), Eq(Var "v", Var "y")))) in
let max_body = Ite(Gt(Var "x",Var "y"), Var "x", Var "y") in
let max = Lam("x", trivial TyInt, None, Lam("y", trivial TyInt, Some max_ret_ty, max_body)) in
type_check [] max None

(declare-fun y () Int)
(declare-fun x () Int)
(declare-fun v () Int)
(assert (= (> x y) true))
(assert (let ((a!1 (not (and (or (> v x) (= v x)) (or (> v y) (= v y))))))
  (and true (= v x) a!1)))



libc++abi: terminating with uncaught exception of type is_non_propositional_predicate::found


## Question 1 ##

Add `Div` to the CoreML AST and make sure the following function type checks. You will have to write the AST for the following function similar to `max` above.

```ocaml
let div = fun (x:int) -> fun (y:{v:int | ¬(v=0)}) -> x/y
```

In [ ]:
(*
 * Fill in here
 *)

## Question 2 ##
Write the CoreML ast for the following function and make sure it type checks:

```ocaml
let rec (sum: k:int -> {v:int | v≥0 ∧ v≥k} = 
  fun (k:int):{v:int | v≥0 ∧ v≥k} -> 
    if k < 0 then 0 
    else let s = sum (k-1) in s + k
```

In [ ]:
(*
 * Fill in here
 *)

## Question 3 ##
Write the CoreML ast for the following function and make sure it type checks:

```ocaml
let rec (fib: n:int -> {v:int | n≤5 ∨ v>n }) = 
  fun (n:int):{v:int | n≤5 ∨ v>n } -> if n<=2 then 1 else fib (n-1) + fib (n-2)
```

In [ ]:
(*
 * Fill in here
 *)

## Question 4 ##
Write the CoreML ast for the following function and make sure it type checks:

```ocaml
let div = fun (x:int) -> fun (y:{v:int | ¬(v=0)}) -> x/y in
let max = fun (x:int) -> fun (y:int) : {v:int | v≥x ∧ v≥y} = if x > y then x else y
let m = 44 in
let n = 23 in
let k = max m n in
div 100 (k-23)
```


In [ ]:
(*
 * Fill in here
 *)